<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="figures/header_small.png">

*Esta libreta contiene material del Taller de Python que se lleva a cabo como parte del 
evento [Data Challenge Industrial 4.0](www.lania.mx/dci). El contenido ha sido adaptado 
por HTM y GED a partir del libro [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) 
de Jake VanderPlas y se mantiene la licencia sobre el texto, 
[CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode), 
y sobre el codigo [MIT license](https://opensource.org/licenses/MIT).*

<!--NAVIGATION-->
< [Introduciendo Scikit-Learn](03.02-Introducing-Scikit-Learn.ipynb) | [Contenido](Index.ipynb) | [Ingeniería de Atributos](03.04-Feature-Engineering.ipynb) >

<a href="https://colab.research.google.com/github/htapia/TallerPythonIntroCienciaDatos/blob/master/notebooks/03.03-Hyperparameters-and-Model-Validation.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>


# Hiperparámetros y validación de modelos

## Validación de modelos

### Como no hacer una validación del modelo 


In [7]:
from sklearn.datasets import load_iris
iris = load_iris()
X = iris.data
y = iris.target

In [8]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=1)

In [9]:
model.fit(X, y)
y_model = model.predict(X)

In [10]:
from sklearn.metrics import accuracy_score
accuracy_score(y, y_model)

1.0

### Forma correcta para la validación del modelo: conjuntos de prueba

In [11]:
from sklearn.cross_validation import train_test_split
# split the data with 50% in each set
X1, X2, y1, y2 = train_test_split(X, y, random_state=0,
                                  train_size=0.5)

# fit the model on one set of data
model.fit(X1, y1)

# evaluate the model on the second set of data
y2_model = model.predict(X2)
accuracy_score(y2, y2_model)

ModuleNotFoundError: No module named 'sklearn.cross_validation'

### Validación cuzada de modelos (cross-validation)

In [ ]:
y2_model = model.fit(X1, y1).predict(X2)
y1_model = model.fit(X2, y2).predict(X1)
accuracy_score(y1, y1_model), accuracy_score(y2, y2_model)

Aquí hay una representación visual de la validación cruzada de cinco veces:

![](figures/05.03-5-fold-CV.png)

In [ ]:
from sklearn.cross_validation import cross_val_score
cross_val_score(model, X, y, cv=5)

In [ ]:
from sklearn.cross_validation import LeaveOneOut
scores = cross_val_score(model, X, y, cv=LeaveOneOut(len(X)))
scores

In [ ]:
scores.mean()

Other cross-validation schemes can be used similarly.
For a description of what is available in Scikit-Learn, use IPython to explore the ``sklearn.cross_validation`` submodule, or take a look at Scikit-Learn's online [cross-validation documentation](http://scikit-learn.org/stable/modules/cross_validation.html).

Otro esquema de validación cruzada puede ser usada de manera similar.
Para obtener una descripción de lo que está disponible en Scikit-Learn, use IPython para explorar el submódulo `` sklearn.cross_validation``, o eche un vistazo a la [documentación de validación cruzada] en línea de Scikit-Learn (http://scikit-learn.org/stable/modules/cross_validation.html).

## Selección del mejor modelo

Of core importance is the following question: *if our estimator is underperforming, how should we move forward?*
There are several possible answers:

- Use a more complicated/more flexible model
- Use a less complicated/less flexible model
- Gather more training samples
- Gather more data to add features to each sample

The answer to this question is often counter-intuitive.
In particular, sometimes using a more complicated model will give worse results, and adding more training samples may not improve your results!
The ability to determine what steps will improve your model is what separates the successful machine learning practitioners from the unsuccessful.


De importancia fundamental es la siguiente pregunta: si nuestro estimador tiene un rendimiento inferior, ¿cómo debemos avanzar? Hay varias respuestas posibles:

     Use un modelo más complicado / más flexible
     Use un modelo menos complicado / menos flexible
     Reúne más muestras de entrenamiento
     Reúna más datos para agregar características a cada muestra

La respuesta a esta pregunta es a menudo contra intuitiva. En particular, a veces usar un modelo más complicado dará peores resultados, ¡y agregar más muestras de entrenamiento puede no mejorar sus resultados! La capacidad de determinar qué pasos mejorarán su modelo es lo que separa a los profesionales exitosos de aprendizaje automático de los que no tuvieron éxito.

### Compensando entre la varianza y la tendencia

Fundamentally, the question of "the best model" is about finding a sweet spot in the tradeoff between *bias* and *variance*.
Consider the following figure, which presents two regression fits to the same dataset:

![](figures/05.03-bias-variance.png)

It is clear that neither of these models is a particularly good fit to the data, but they fail in different ways.

The model on the left attempts to find a straight-line fit through the data.
Because the data are intrinsically more complicated than a straight line, the straight-line model will never be able to describe this dataset well.
Such a model is said to *underfit* the data: that is, it does not have enough model flexibility to suitably account for all the features in the data; another way of saying this is that the model has high *bias*.

The model on the right attempts to fit a high-order polynomial through the data.
Here the model fit has enough flexibility to nearly perfectly account for the fine features in the data, but even though it very accurately describes the training data, its precise form seems to be more reflective of the particular noise properties of the data rather than the intrinsic properties of whatever process generated that data.
Such a model is said to *overfit* the data: that is, it has so much model flexibility that the model ends up accounting for random errors as well as the underlying data distribution; another way of saying this is that the model has high *variance*.

Está claro que ninguno de estos modelos se ajusta particularmente bien a los datos, pero fallan de diferentes maneras.

El modelo de la izquierda intenta encontrar un ajuste en línea recta a través de los datos. Debido a que los datos son intrínsecamente más complicados que una línea recta, el modelo de línea recta nunca podrá describir bien este conjunto de datos. Se dice que un modelo de este tipo no ajusta los datos: es decir, no tiene suficiente flexibilidad del modelo para dar cuenta de manera adecuada de todas las características de los datos; Otra forma de decir esto es que el modelo tiene un alto sesgo.

El modelo de la derecha intenta ajustar un polinomio de alto orden a través de los datos. Aquí, el ajuste del modelo tiene suficiente flexibilidad para dar cuenta casi a la perfección de las características finas en los datos, pero a pesar de que describe con mucha precisión los datos de entrenamiento, su forma precisa parece reflejar más las propiedades de ruido particulares de los datos en lugar de lo intrínseco. propiedades de cualquier proceso que haya generado esos datos. Se dice que dicho modelo sobreajusta los datos: es decir, tiene tanta flexibilidad que el modelo termina teniendo en cuenta los errores aleatorios, así como la distribución de datos subyacente; Otra forma de decir esto es que el modelo tiene una gran *varianza*.

Para ver esto desde otra perspectiva, considere lo que sucede sí usamos estos dos modelos para predecir el valor de y para algunos datos nuevos.
En los siguientes diagramas, los puntos rojo / más claro indican datos que se omiten del conjunto de entrenamiento:

![](figures/05.03-bias-variance-2.png)

La puntuación aquí es la puntuación $R^2$, o [coeficiente de determinación] (https://en.wikipedia.org/wiki/Coefficient_of_determination), que mide qué tan bien se desempeña un modelo en relación con una media simple de los valores objetivo . $R^2=1$ indica una coincidencia perfecta, $R^2=0$ indica que el modelo no es mejor que simplemente tomar la media de los datos, y los valores negativos significan modelos aún peores.
A partir de los puntajes asociados con estos dos modelos, podemos hacer una observación más general:

- Para los modelos de alto sesgo, el rendimiento del modelo en el conjunto de validación es similar al rendimiento en el conjunto de entrenamiento.
- Para los modelos de alta varianza, el rendimiento del modelo en el conjunto de validación es mucho peor que el rendimiento en el conjunto de entrenamiento.

Sí imaginamos que tenemos alguna capacidad para ajustar la complejidad del modelo, esperaríamos que el puntaje de entrenamiento y el puntaje de validación se comporten como se ilustra en la siguiente figura:

![](figures/05.03-validation-curve.png)
[figure source in Appendix](06.00-Figure-Code.ipynb#Validation-Curve)

El diagrama que se muestra aquí a menudo se llama * curva de validación *, y vemos las siguientes características esenciales:

- El puntaje de entrenamiento es en todas partes más alto que el puntaje de validación. Este es generalmente el caso: el modelo se ajustará mejor a los datos que ha visto que a los datos que no ha visto.
- Para una complejidad de modelo muy baja (un modelo de alto sesgo), los datos de entrenamiento no están ajustados, lo que significa que el modelo es un mal predictor tanto para los datos de entrenamiento como para los datos no vistos previamente.
- Para una complejidad de modelo muy alta (un modelo de alta varianza), los datos de entrenamiento están sobreajustados, lo que significa que el modelo predice muy bien los datos de entrenamiento, pero falla para cualquier dato no visto previamente.
- Para algunos valores intermedios, la curva de validación tiene un máximo. Este nivel de complejidad indica una compensación adecuada entre sesgo y varianza.

Los medios para ajustar la complejidad del modelo varían de un modelo a otro; Cuando analicemos los modelos individuales en profundidad en secciones posteriores, veremos cómo cada modelo permite tal ajuste.

### Curvas de validación en Scikit-Learn

Veamos un ejemplo de uso de validación cruzada para calcular la curva de validación para una clase de modelos.
Aquí usaremos un modelo de * regresión polinómica *: este es un modelo lineal generalizado en el cual el grado del polinomio es un parámetro ajustable.
Por ejemplo, un polinomio de grado 1 ajusta una línea recta a los datos; para los parámetros del modelo $ a $ y $ b $:

$$
y = ax + b
$$

Un polinomio de grado 3 ajusta una curva cúbica a los datos; para los parámetros del modelo $ a, b, c, d $:

$$
y = ax ^ 3 + bx ^ 2 + cx + d
$$

Podemos generalizar esto a cualquier número de características polinómicas.
En Scikit-Learn, podemos implementar esto con una regresión lineal simple combinada con el preprocesador polinómico.
Usaremos una * tubería * para unir estas operaciones (discutiremos las características y tuberías polinomiales más detalladamente en [Ingeniería de características] (05.04-Feature-Engineering.ipynb)):

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

def PolynomialRegression(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree),
                         LinearRegression(**kwargs))

In [ ]:
import numpy as np

def make_data(N, err=1.0, rseed=1):
    # randomly sample the data
    rng = np.random.RandomState(rseed)
    X = rng.rand(N, 1) ** 2
    y = 10 - 1. / (X.ravel() + 0.1)
    if err > 0:
        y += err * rng.randn(N)
    return X, y

X, y = make_data(40)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn; seaborn.set()  # plot formatting

X_test = np.linspace(-0.1, 1.1, 500)[:, None]

plt.scatter(X.ravel(), y, color='black')
axis = plt.axis()
for degree in [1, 3, 5]:
    y_test = PolynomialRegression(degree).fit(X, y).predict(X_test)
    plt.plot(X_test.ravel(), y_test, label='degree={0}'.format(degree))
plt.xlim(-0.1, 1.0)
plt.ylim(-2, 12)
plt.legend(loc='best');

In [ ]:
from sklearn.learning_curve import validation_curve
degree = np.arange(0, 21)
train_score, val_score = validation_curve(PolynomialRegression(), X, y,
                                          'polynomialfeatures__degree', degree, cv=7)

plt.plot(degree, np.median(train_score, 1), color='blue', label='training score')
plt.plot(degree, np.median(val_score, 1), color='red', label='validation score')
plt.legend(loc='best')
plt.ylim(0, 1)
plt.xlabel('degree')
plt.ylabel('score');

In [ ]:
plt.scatter(X.ravel(), y)
lim = plt.axis()
y_test = PolynomialRegression(3).fit(X, y).predict(X_test)
plt.plot(X_test.ravel(), y_test);
plt.axis(lim);

## Curvas de aprendizaje

In [ ]:
X2, y2 = make_data(200)
plt.scatter(X2.ravel(), y2);

In [ ]:
degree = np.arange(21)
train_score2, val_score2 = validation_curve(PolynomialRegression(), X2, y2,
                                            'polynomialfeatures__degree', degree, cv=7)

plt.plot(degree, np.median(train_score2, 1), color='blue', label='training score')
plt.plot(degree, np.median(val_score2, 1), color='red', label='validation score')
plt.plot(degree, np.median(train_score, 1), color='blue', alpha=0.3, linestyle='dashed')
plt.plot(degree, np.median(val_score, 1), color='red', alpha=0.3, linestyle='dashed')
plt.legend(loc='lower center')
plt.ylim(0, 1)
plt.xlabel('degree')
plt.ylabel('score');

Las líneas continuas muestran los nuevos resultados, mientras que las líneas discontinuas más débiles muestran los resultados del conjunto de datos anterior más pequeño.
A partir de la curva de validación, es claro que el conjunto de datos más grande puede admitir un modelo mucho más complicado: el pico aquí es probablemente de alrededor de un grado de 6, pero incluso un modelo de grado 20 no está ajustando demasiado los datos: la validación y la capacitación los puntajes permanecen muy cerca.

Por lo tanto, vemos que el comportamiento de la curva de validación no tiene una sino dos entradas importantes: la complejidad del modelo y el número de puntos de entrenamiento.
A menudo es útil explorar el comportamiento del modelo en función del número de puntos de entrenamiento, lo que podemos hacer utilizando subconjuntos de datos cada vez más grandes para adaptarnos a nuestro modelo.
Una gráfica del puntaje de entrenamiento / validación con respecto al tamaño del conjunto de entrenamiento se conoce como * curva de aprendizaje. *

El comportamiento general que esperaríamos de una curva de aprendizaje es este:

- Un modelo de una complejidad dada * sobreajustará * un pequeño conjunto de datos: esto significa que el puntaje de entrenamiento será relativamente alto, mientras que el puntaje de validación será relativamente bajo.
- Un modelo de una complejidad dada * subestimará * un gran conjunto de datos: esto significa que el puntaje de entrenamiento disminuirá, pero el puntaje de validación aumentará.
- Un modelo nunca, excepto por casualidad, dará una mejor puntuación al conjunto de validación que al conjunto de entrenamiento: esto significa que las curvas deben seguir acercándose pero nunca cruzarse.

Con estas características en mente, esperaríamos que una curva de aprendizaje se vea cualitativamente como la que se muestra en la siguiente figura:

![](figures/05.03-learning-curve.png)
[figure source in Appendix](06.00-Figure-Code.ipynb#Learning-Curve)

La característica notable de la curva de aprendizaje es la convergencia a un puntaje particular a medida que crece el número de muestras de entrenamiento.
En particular, una vez que tenga suficientes puntos para que un modelo en particular haya convergido, * ¡agregar más datos de entrenamiento no lo ayudará! *
La única forma de aumentar el rendimiento del modelo en este caso es usar otro modelo (a menudo más complejo).

### Curvas de aprendizaje en Scikit-Learn

La característica notable de la curva de aprendizaje es la convergencia a un puntaje particular a medida que crece el número de muestras de entrenamiento.
En particular, una vez que tenga suficientes puntos para que un modelo en particular haya convergido, * ¡agregar más datos de entrenamiento no lo ayudará! *
La única forma de aumentar el rendimiento del modelo en este caso es usar otro modelo (a menudo más complejo).

In [ ]:
from sklearn.learning_curve import learning_curve

fig, ax = plt.subplots(1, 2, figsize=(16, 6))
fig.subplots_adjust(left=0.0625, right=0.95, wspace=0.1)

for i, degree in enumerate([2, 9]):
    N, train_lc, val_lc = learning_curve(PolynomialRegression(degree),
                                         X, y, cv=7,
                                         train_sizes=np.linspace(0.3, 1, 25))

    ax[i].plot(N, np.mean(train_lc, 1), color='blue', label='training score')
    ax[i].plot(N, np.mean(val_lc, 1), color='red', label='validation score')
    ax[i].hlines(np.mean([train_lc[-1], val_lc[-1]]), N[0], N[-1],
                 color='gray', linestyle='dashed')

    ax[i].set_ylim(0, 1)
    ax[i].set_xlim(N[0], N[-1])
    ax[i].set_xlabel('training size')
    ax[i].set_ylabel('score')
    ax[i].set_title('degree = {0}'.format(degree), size=14)
    ax[i].legend(loc='best')

## Validación en la práctica: Grid Search

La discusión anterior tiene la intención de darle una cierta intuición sobre el equilibrio entre sesgo y varianza, y su dependencia de la complejidad del modelo y el tamaño del conjunto de entrenamiento.
En la práctica, los modelos generalmente tienen más de una perilla para girar y, por lo tanto, las curvas de validación y las curvas de aprendizaje cambian de líneas a superficies multidimensionales.
En estos casos, tales visualizaciones son difíciles y preferimos simplemente encontrar el modelo particular que maximice la puntuación de validación.

Scikit-Learn proporciona herramientas automatizadas para hacer esto en el módulo de búsqueda de cuadrícula.
Aquí hay un ejemplo del uso de la búsqueda de cuadrícula para encontrar el modelo polinómico óptimo.
Exploraremos una cuadrícula tridimensional de características del modelo; a saber, el grado polinomial, la bandera que nos indica si debemos ajustar la intersección, y la bandera que nos dice si debemos normalizar el problema.
Esto se puede configurar utilizando el metaestimulador `` GridSearchCV`` de Scikit-Learn:

In [ ]:
from sklearn.grid_search import GridSearchCV

param_grid = {'polynomialfeatures__degree': np.arange(21),
              'linearregression__fit_intercept': [True, False],
              'linearregression__normalize': [True, False]}

grid = GridSearchCV(PolynomialRegression(), param_grid, cv=7)

In [ ]:
grid.fit(X, y);

In [ ]:
grid.best_params_

In [ ]:
model = grid.best_estimator_

plt.scatter(X.ravel(), y)
lim = plt.axis()
y_test = model.fit(X, y).predict(X_test)
plt.plot(X_test.ravel(), y_test, hold=True);
plt.axis(lim);

## Summary

En ésta sección exploramos el concepto de validación de modelo y optimización de hiperparámetros, enfocados en los aspectos de tendencia y varianza de los datos y su efecto en el modelado. El uso de conjuntos de validación o validación cruzada son *fundamentales* para obtener los mejores parámetros y evitar el efecto de un sobre ajuste del modelo, lo cual permite explorar el uso de modelos más complejos o flexibles.

<!--NAVIGATION-->
< [Introduciendo Scikit-Learn](03.02-Introducing-Scikit-Learn.ipynb) | [Contenido](Index.ipynb) | [Ingeniería de Atributos](03.04-Feature-Engineering.ipynb) >

<a href="https://colab.research.google.com/github/htapia/TallerPythonIntroCienciaDatos/blob/master/notebooks/03.03-Hyperparameters-and-Model-Validation.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>
